In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Estudiantes:

**Jose Andres Henao Alzate**



**Jhon Eduar García Ortiz**

# Embedded ML - Lab 2.3: TensorFlow Lite Micro

Tensor Flow Lite Micro (TFLM) is a library that aims to run ML models efficiently on embedded systems. It's a C++ library that provides a version of the TensorFlow Lite interpreter that supports less types of operations and uses less memory. The library also provides helper functions for data pre- and post-processing.

### Learning outcomes


* Explain the basic concepts associated with TFLM
* Use the API to implement the TFLM workflow for an embedded application
* Execute TFLM code on a microcontroller-based embedded system

### TensorFlow Lite Micro workflow

TFLM's high-level workflow is rather simple:
* Generate a small TensorFlow model that can fit your target device and contains supported operations.
* Convert to a TensorFlow Lite model using the TensorFlow Lite converter, applying quantization if required.
* Convert to a C byte array using standard tools and stored it in the read-only program memory on device.
* Run inference on device using the TFLM C++ library and process the results.

### Hello World and Hello Human

After installing the Arduino IDE and the board files, you should install the Harvard_TinyMLx library that contains the TensorFlow Lite Micro and other resources and examples to build ML apps with Arduino and TFLM. Later on, depending on the application you want to build and the specific hardware to be used, you should install the propper peripheral drivers for communication, sensing and actuating.



*   Install Arduino IDE 2 from: https://downloads.arduino.cc/arduino-ide/arduino-ide_2.3.2_Linux_64bit.AppImage
*   From the boards manager install: Arduino Mbed OS Nano boards
*   Allow the linux user to access serial port: `sudo usermod -a -G dialout \<username\>` (reboot afterwards)
*   From the library manager install: Harvard_TinyMLx




Now open the **Hello World** example from the Harvard_TinyMLx library File->Examples->Harvard_TinyMLx in Arduino IDE (also available in [this repo](https://github.com/tinyMLx/arduino-library/tree/main/examples/hello_world)), compile it and run it on the microcontroller board. It is an ML model to predict a sine wave that is used to dim on and off an LED. The Arduino IDE serial monitor should also show interger numbers up and down trying to model a sine wave. This is a test app to make sure that the basic HW and SW elements, including TFLM, are working.

Inspect the code to make sure you identify and understand the main parts of the workflow.

Running on-device inference using the TFLM C++ library usually involves:

* Include the library headers
* Include the model header
* Load a model
* Instantiate operations resolver
* Allocate memory
* Instantiate interpreter
* Read and pre-process input data
* Provide inputs to the allocated tensors
* Run inference
* Get results from the output tensors
* Take action based on outputs

After you have succesfully run the Hello World example, move on to running the **Person Detection** example from the same library. Explore the code in detail to understand how to handle the **camera**.

### TinyML application development

ML applications that run on embeded systems with very limited resources are often called TinyML. In this lab the goal is to develop a simple TinyML application that uses computer vision up to its deployment on the target embedded device: **Arduino Nano 33 BLE.**

Follow these steps in order to develop your TinyML application:

1. Select two visual objects that are radically different and  assemble a dataset that contains at least hundreds or thousands of examples. You can create the images yourslef or extract them from a public database and apply data augmentation.

2. Design and train a model to classify between the two chosen objects. You can build a dense or CNN model from scratch, or use transfer learning, but you should always keep in mind the very limited memory resources of the target device as well as the image properties of the embedded camera.

3.   Export the trained model to a file and convert it to a C header by running the following linux command: `xxd -i converted_model.tflite > converted_model_data.h`

4.   Develop an Arduino code based on the Hello World and Person Detection examples, to detect whether any of the two objects are present on the camera view. Indicate the result through the RGB LED.

Include in your notebook submission both the code you developed to build the model as well as the C++ codes for the MCU.

In [ ]:
# Conecta Google Drive para acceder a los archivos almacenados allí
from google.colab import drive
drive.mount('/content/drive')

# Importación de librerías necesarias para el procesamiento y entrenamiento
import os
import numpy as np
import cv2
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models

# Parámetros iniciales
IMG_SIZE = (100, 100)  # Tamaño al que se redimensionarán las imágenes
base_path = '/content/drive/MyDrive/EML_2_3'  # Ruta base en Google Drive
categories = ['banano', 'manzana']  # Clases o etiquetas que se usarán

# Función para cargar las imágenes desde carpetas según su etiqueta
def cargar_datos(path):
    X = []  # Lista para almacenar las imágenes
    y = []  # Lista para almacenar las etiquetas correspondientes
    for label in categories:
        carpeta = os.path.join(path, label)  # Ruta a la carpeta de cada clase
        for nombre_img in os.listdir(carpeta):  # Recorre cada archivo en la carpeta
            ruta_img = os.path.join(carpeta, nombre_img)
            img = cv2.imread(ruta_img, cv2.IMREAD_GRAYSCALE)  # Carga la imagen en escala de grises
            if img is not None:
                img = cv2.resize(img, IMG_SIZE)  # Redimensiona la imagen
                X.append(img)  # Añade la imagen a la lista
                y.append(label)  # Añade su etiqueta correspondiente
    return np.array(X), np.array(y)

# Carga las imágenes de entrenamiento y prueba desde las carpetas correspondientes
X_train, y_train = cargar_datos(os.path.join(base_path, 'train'))
X_test, y_test = cargar_datos(os.path.join(base_path, 'test'))

# Muestra cuántas imágenes se han cargado en total
print(f"Datos cargados: {X_train.shape[0]} imágenes de entrenamiento, {X_test.shape[0]} de prueba")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 Datos cargados: 472 imágenes de entrenamiento, 92 de prueba


In [ ]:
# Normalización y reshaping
X_train = X_train / 255.0
X_test = X_test / 255.0
X_train = X_train.reshape(-1, 100, 100, 1)
X_test = X_test.reshape(-1, 100, 100, 1)


In [ ]:
from tensorflow.keras.optimizers import Adam

# Codificación de etiquetas: convierte las clases 'banano' y 'manzana' a valores numéricos (0 y 1)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train)  # Ajusta y transforma las etiquetas de entrenamiento
y_test = le.transform(y_test)        # Transforma las etiquetas de prueba con el mismo codificador

# Definición del modelo CNN (Red Neuronal Convolucional)
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(3, (3, 3), activation='relu', input_shape=(100, 100, 1)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(2, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
# Compilación del modelo: se especifica el optimizador, la función de pérdida y las métricas a seguir
model.compile(
    optimizer='Adam',                 # Optimizador Adam (eficiente y muy utilizado)
    loss='binary_crossentropy',      # Pérdida para clasificación binaria
    metrics=['accuracy']             # Métrica a seguir durante entrenamiento y evaluación
)

# Entrenamiento del modelo con las imágenes de entrenamiento
history = model.fit(
    X_train, y_train,                # Datos de entrada y sus etiquetas
    epochs=50,                       # Número de épocas de entrenamiento
    batch_size=4,                    # Tamaño del lote (cuántas imágenes procesa por paso)
    validation_data=(X_test, y_test) # Datos de validación para evaluar el rendimiento en cada época
)

# Evaluación final con el conjunto de prueba
loss, acc = model.evaluate(X_test, y_test)
print(f'Precisión en test: {acc:.2f}')  # Imprime la precisión obtenida


Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


118/118 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.5171 - loss: 0.6908 - val_accuracy: 0.6522 - val_loss: 0.6895
Epoch 2/50
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5182 - loss: 0.6910 - val_accuracy: 0.4674 - val_loss: 0.6874
Epoch 3/50
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5640 - loss: 0.6896 - val_accuracy: 0.5217 - val_loss: 0.6818
Epoch 4/50
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6546 - loss: 0.6834 - val_accuracy: 0.5978 - val_loss: 0.6795
Epoch 5/50
118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6502 - loss: 0.6794 - val_accuracy: 0.5978 - val_loss: 0.6772
Epoch 6/50
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5805 - loss: 0.6758 - val_accuracy: 0.6304 - val_loss: 0.6739
Epoch 7/50
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6528 - loss: 0.6775 - val_accuracy: 0.5761 - val_loss: 0.6701
Epoch 8/50
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6394 - loss: 0.6816 - val_accuracy: 0.5543 - val_

In [ ]:
print(np.unique(y_train, return_counts=True))


(array([0, 1]), array([228, 244]))


In [ ]:
model.save('ManzanaBananos_best_liviano.keras')

#Generación del archivo lite.

In [ ]:
import pathlib

# Ruta del modelo previamente entrenado y guardado en formato .keras
ruta_modelo = 'ManzanaBananos_best_liviano.keras'

# Cargar el modelo desde el archivo .keras
modelo = tf.keras.models.load_model(ruta_modelo)

# Crear un convertidor para transformar el modelo Keras a formato TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(modelo)

# Realizar la conversión a TensorFlow Lite
tflite_model = converter.convert()

# Definir la ruta de salida y guardar el archivo .tflite generado
ruta_salida = pathlib.Path('ManzanasBananos_best_liviano.tflite')
ruta_salida.write_bytes(tflite_model)


Saved artifact at '/tmp/tmp7_7ow98q'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 100, 100, 1), dtype=tf.float32, name='input_layer_2')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  136236641400016: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136236641400784: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136236641399440: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136236926006672: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136236641400592: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136236641401936: TensorSpec(shape=(), dtype=tf.resource, name=None)


3468

In [ ]:
import tensorflow as tf
import numpy as np

# Cargar el modelo entrenado previamente desde un archivo .keras
modelo = tf.keras.models.load_model("/content/ManzanaBananos_best_liviano.keras")

# Crear el convertidor de TensorFlow Lite a partir del modelo cargado
converter = tf.lite.TFLiteConverter.from_keras_model(modelo)

# Activar la optimización para reducir tamaño y hacer el modelo más eficiente
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Definir un conjunto representativo de datos para ayudar en la cuantización
# Este paso es clave para que el modelo entienda el rango de los datos reales
def representative_dataset():
    for i in range(300):
        image = X_train[i].astype(np.float32)  # Asegura tipo de dato float32
        image = np.expand_dims(image, axis=0)  # Cambia de (100,100,1) a (1,100,100,1)
        yield [image]

# Asignar el dataset representativo al convertidor
converter.representative_dataset = representative_dataset

# Configurar la cuantización completa a entero (INT8)
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8     # La entrada será int8
converter.inference_output_type = tf.int8    # La salida será int8

# Realizar la conversión a TFLite cuantizado
tflite_model_quant = converter.convert()

# Guardar el modelo resultante como archivo .tflite
with open("/content/ManzanaBananos_best_INT8_liviano.tflite", "wb") as f:
    f.write(tflite_model_quant)



Saved artifact at '/tmp/tmp0pw_v8aj'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 100, 100, 1), dtype=tf.float32, name='input_layer_2')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  136236007477840: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136236007479184: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136236007479760: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136236007480336: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136236641395024: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136236641401360: TensorSpec(shape=(), dtype=tf.resource, name=None)


/usr/local/lib/python3.11/dist-packages/tensorflow/lite/python/convert.py:997: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


In [ ]:
import os

ruta_tflite = 'ManzanasBananos_best.tflite'

tamaño_bytes = os.path.getsize(ruta_tflite)
tamaño_kb = tamaño_bytes / 1024

print(f"El archivo '{ruta_tflite}' pesa aproximadamente {tamaño_kb:.2f} KB.")


El archivo 'ManzanasBananos_best.tflite' pesa aproximadamente 20.00 KB.


In [ ]:
import os

ruta_tflite = 'ManzanaBananos_best_INT8.tflite'

tamaño_bytes = os.path.getsize(ruta_tflite)
tamaño_kb = tamaño_bytes / 1024

print(f"El archivo '{ruta_tflite}' pesa aproximadamente {tamaño_kb:.2f} KB.")


El archivo 'ManzanaBananos_best_INT8.tflite' pesa aproximadamente 10.38 KB.


#Generación del array de C++ para el sistema sin cuantización.

In [ ]:
import os
from google.colab import files


tflite_filename = "ManzanasBananos_best_liviano.tflite"

# Verificar existencia
if not os.path.exists(tflite_filename):
    raise FileNotFoundError(f" El archivo '{tflite_filename}' no se encuentra en el entorno de trabajo de Colab.")


print(" Instalando 'xxd'...")
os.system("apt-get install xxd -y")

# Convertir a model.h
print(" Convirtiendo a 'model_best.h'...")
os.system(f"xxd -i {tflite_filename} > model_best_liviano.h")

# Descargar model.h
print("⬇ Descargando 'model.h'...")
files.download("model_best_liviano.h")



 Instalando 'xxd'...
 Convirtiendo a 'model_best.h'...
⬇ Descargando 'model.h'...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#Generación del array de C++ para el sistema con cuantización.

In [ ]:
import os
from google.colab import files


tflite_filename = "ManzanaBananos_best_INT8.tflite"

# Verificar existencia
if not os.path.exists(tflite_filename):
    raise FileNotFoundError(f" El archivo '{tflite_filename}' no se encuentra en el entorno de trabajo de Colab.")


print(" Instalando 'xxd'...")
os.system("apt-get install xxd -y")

# Convertir a model.h
print(" Convirtiendo a 'model_INT8.h'...")
os.system(f"xxd -i {tflite_filename} > model_INT8.h")

# Descargar model.h
print("⬇ Descargando 'model.h'...")
files.download("model_INT8.h")



 Instalando 'xxd'...
 Convirtiendo a 'model_INT8.h'...
⬇ Descargando 'model.h'...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

* Lo anterior se resume en la siguiente tabla.

In [ ]:
import pandas as pd
from IPython.display import display, HTML

# Datos de los modelos descritos
data = {
    "Característica": [
        "Arquitectura",
        "N° Filtros (Conv2D)",
        "Capas Densas",
        "Formato Final",
        "Dataset",
        "Cantidad de Imágenes",
        "Precisión Entrenamiento",
        "Precisión Prueba",
        "Ventaja Principal",
        "Desventaja Principal"
    ],
    "Modelo 1 (CNN Simple)": [
        "Conv2D(3) → MaxPool → Dense(2) → Dense(1)",
        "3",
        "2 y 1",
        "float32 (sin cuantizar)",
        "Amplio, público (≈1000 imágenes)",
        "≈ 1000",
        "98%",
        "98%",
        "Alto rendimiento en posiciones conocidas",
        "Sobreentrenamiento por baja variabilidad"
    ],
    "Modelo 2 (CNN Cuantizado)": [
        "Conv2D(16) → MaxPool → Dense(20) → Dense(1)",
        "16",
        "20 y 1",
        "INT8 (cuantizado)",
        "Propio, pequeño (menos imágenes)",
        "< 1000",
        "70%",
        "63%",
        "Compatible con microcontrolador",
        "Menor precisión por escasez de datos"
    ]
}

# Crear DataFrame
df = pd.DataFrame(data)

# Mostrar tabla HTML con estilo
display(HTML(df.to_html(index=False, classes='table table-bordered table-hover', border=1)))


Característica,Modelo 1 (CNN Simple),Modelo 2 (CNN Cuantizado)
Arquitectura,Conv2D(3) → MaxPool → Dense(2) → Dense(1),Conv2D(16) → MaxPool → Dense(20) → Dense(1)
N° Filtros (Conv2D),3,16
Capas Densas,2 y 1,20 y 1
Formato Final,float32 (sin cuantizar),INT8 (cuantizado)
Dataset,"Amplio, público (≈1000 imágenes)","Propio, pequeño (menos imágenes)"
Cantidad de Imágenes,≈ 1000,< 1000
Precisión Entrenamiento,98%,70%
Precisión Prueba,98%,63%
Ventaja Principal,Alto rendimiento en posiciones conocidas,Compatible con microcontrolador
Desventaja Principal,Sobreentrenamiento por baja variabilidad,Menor precisión por escasez de datos


### Evaluación y Comparación de Modelos
En la búsqueda del modelo óptimo para la aplicación, se evaluaron dos arquitecturas de redes neuronales convolucionales (CNN) con enfoques distintos en cuanto a complejidad, optimización y cantidad de datos utilizados.

El primer modelo, de arquitectura sencilla (una capa Conv2D con 3 filtros, seguida de MaxPooling y dos capas densas), fue entrenado con un dataset amplio de aproximadamente 1000 imágenes. Alcanzó una precisión del 98% tanto en entrenamiento como en prueba, pero evidenció sobreentrenamiento, debido a la alta similitud entre las imágenes del conjunto de entrenamiento y validación, lo que limitó su capacidad de generalización a nuevas posiciones o variaciones.

El segundo modelo, más profundo (una Conv2D con 16 filtros y una capa densa intermedia de 20 neuronas), fue cuantizado a formato INT8 para ser ejecutable en un microcontrolador con restricciones de memoria y procesamiento. Este fue entrenado con un dataset propio, más pequeño pero ligeramente más diverso, obteniendo una precisión del 70% en entrenamiento y 63% en prueba. Su rendimiento inferior se atribuye principalmente a la escasez y limitada variabilidad del dataset.

Cabe destacar que, en la fase de inferencia, el modelo entrenado con mayor cantidad de imágenes (aunque más simple) mostró un desempeño más sólido y consistente. Esto evidencia que un mayor volumen de datos puede compensar, en cierta medida, la simplicidad arquitectónica, mientras que una red más compleja no garantiza mejores resultados si se entrena con un conjunto de datos limitado o poco representativo.


# Conclusiones

* En conclusión, los resultados del laboratorio demostraron que modelos convolucionales simples pueden superar a modelos más profundos, sin embargo, esto depende fuertemente de datos de entrenamiento de alta calidad, representativos, diversos y con bajo nivel de ruido. Aunque el primer modelo alcanzó una mayor precisión, su capacidad de generalización se vio afectada por el sobreentrenamiento derivado de la similitud entre imágenes. Por su parte, el segundo modelo, optimizado mediante cuantización para su uso en hardware embebido, mostró un rendimiento inferior debido principalmente a la baja calidad y cantidad del dataset. Estos hallazgos evidencian que el desempeño de un modelo no depende exclusivamente de su arquitectura, sino que está fuertemente condicionado por las características de los datos utilizados para entrenarlo.

* Por otra parte, se pudo evidenciar la importancia del proceso de cuantización para lograr la reducción de los recursos de cómputo necesarios para la ejecución de modelos moderadamente profundos. Esto porque se pudo ejecutar, en un kit Arduino 33 BLE, un modelo que inicialmente usaba representacion de float32, pero se optimizó su ocupación en memoria por medio de la cuantización a enteros sin signo de 8 bits.

* Esta práctica evidencia el constante intercambio que se debe llevar a cabo para lograr un modelo tanto generalizable como preciso, esto se refiere al equilibrio entre la complejidad en la arquitectura, la representación de sus datos en memoria y, en algunos casos, la latencia en la inferencia según la aplicación.



